In [ ]:
__author__ = 'Benjamin Weaver <benjamin.weaver@noirlab.edu>, Alice Jacques <alice.jacques@noirlab.edu>, Astro Data Lab Team <datalab@noirlab.edu>'
__version__ = '20230614'
__datasets__ = ['desi_edr']
__keywords__ = ['query', 'DESI']

# How to Query DESI EDR Data

## Table of Contents

* [Goals](#Goals)
* [Summary](#Summary)
* [Disclaimer and Attribution](#Disclaimer-and-Attribution)
* [Initial Setup](#Initial-Setup)
* [Contents of the Database](#Contents-of-the-Database)
* [Authentication](#Authentication)
* [Simple Queries](#Simple-Queries)
* [Joining Tables](#Joining-Tables)
* [Matching Redshift Tables to Targeting Data](#Matching-Redshift-Tables-to-Targeting-Data)
* [Survey Progress](#Survey-Progress)
* [Using q3c](#Using-q3c)
* [Resources and References](#Resources-and-References)

## Goals

Demonstrate a variety of queries using the `desi_edr` data set.

## Summary

This notebook will cover the basics of using the DESI spectroscopic production database, `desi_edr`, which is loaded from the outputs of the DESI pipeline. This notebook is intended to cover as much of the detail of DESI's [introductory notebook tutorial](https://github.com/desihub/tutorials/blob/main/database/spectroscopic-production-database.ipynb) as possible, although rewritten for the Data Lab environment. For more details, including detailed table descriptions see DESI's [database access page](https://data.desi.lbl.gov/doc/access/database/).

## Disclaimer and Attribution

If you use this notebook for your published science, please acknowledge the following:

* DESI Data citation and acknowledgments: https://data.desi.lbl.gov/doc/acknowledgments/
* Data Lab concept paper: Fitzpatrick *et al.*, "The NOAO Data Laboratory: a conceptual overview", SPIE, **9149**, 2014, http://dx.doi.org/10.1117/12.2057445
* Data Lab disclaimer: https://datalab.noirlab.edu/disclaimers.php

## Initial Setup

This just imports everything we need and sets up paths and environment variables so we can find things.

In [ ]:
#
# Imports
#
import csv
import os
from io import StringIO
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.font_manager import fontManager, FontProperties
import astropy.units as u
from dl import queryClient as qc, storeClient as sc, authClient as ac
#
# DESI software
#
from desitarget import __version__ as desitarget_version
from desitarget.targetmask import (desi_mask, mws_mask, bgs_mask)
#
# from astropy.time import Time
# from astropy.coordinates import ICRS
print(f"desitarget=={desitarget_version}")

## Contents of the Database

### Schema

All tables are grouped into a database *schema* and that schema is named for the production run, (*e.g.* `fuji`).  When writing "raw" SQL, table names need to be schema-qualified, for example, `fuji.target`.  However, the SQLAlchemy abstraction layer is designed to take care of this for you.

### Important notes

* This database does not contain any sky spectra. Both deliberately-targeted (`targetid & 2**59 != 0`) and negative targetid (`targetid < 0`) sky spectra are excluded.
* Only quantities derived from *cumulative* tile-based spectra are included at the present time.

### The tables

* `photometry`. This contains the pure photometric data. Usually this is derived from the LS DR9 Tractor data, but not every *targeted* object has Tractor photometry.
  - Loaded from `tractorphot` files in the [lsdr9-photometry VAC](https://data.desi.lbl.gov/doc/releases/edr/vac/lsdr9-photometry/): `/global/cfs/cdirs/desi/public/edr/vac/edr/lsdr9-photometry/fuji/v2.1/potential-targets/tractorphot/tractorphot-potential-*-fuji.fits`.
  - SQLAlchemy object: `db.Photometry`.
  - Primary key: `targetid`.
  - [Column descriptions](https://data.desi.lbl.gov/doc/access/database/#photometry).
* `target`. This contains the targeting bits and other data generated by `desitarget`.
  - Loaded from `targetphot` file in the [lsdr9-photometry VAC](https://data.desi.lbl.gov/doc/releases/edr/vac/lsdr9-photometry/): `/global/cfs/cdirs/desi/public/edr/vac/edr/lsdr9-photometry/fuji/v2.1/potential-targets/targetphot-potential-fuji.fits`.
  - SQLAlchemy object: `db.Target`.
  - Unique identifier: (`targetid`, `survey`, `tileid`).
  - Primary key: `id`, a unique, arbitrary integer composed from (`targetid`, `survey`, `tileid`).
  - [Column descriptions](https://data.desi.lbl.gov/doc/access/database/#target).
* `tile`. This contains information about observations grouped by tile.
  - Loaded from top-level `tiles-fuji.fits`.
  - SQLAlchemy object: `db.Tile`.
  - Primary key: `tileid`.
  - [Column descriptions](https://data.desi.lbl.gov/doc/access/database/#tile).
* `exposure`. This contains information about individual exposures.
  - Loaded from top-level `exposures-fuji.fits`, `EXPOSURES` HDU.
  - SQLAlchemy object: `db.Exposure`.
  - Primary key: `expid`.
  - [Column descriptions](https://data.desi.lbl.gov/doc/access/database/#exposure).
* `frame`. This contains information about individual exposures, but broken down by camera.  There will usually, but not always, be 30 frames per exposure.
  - Loaded from top-level `exposures-fuji.fits`, `FRAMES` HDU.
  - SQLAlchemy object: `db.Frame`.
  - Unique identifier: (`expid`, `camera`).
  - Primary key: `frameid`, composed from `expid` and a mapping of `camera` to an arbitrary integer.
  - [Column descriptions](https://data.desi.lbl.gov/doc/access/database/#frame).
* `fiberassign`. This contains information about fiber positions.
  - Loaded from fiberassign files in the [tiles product](https://data.desi.lbl.gov/public/edr/target/fiberassign/tiles/tags/0.5/).  All fiberassign files corresponding to tiles in the `tile` table are loaded.
  - SQLAlchemy object: `db.Fiberassign`.
  - Unique identifier: (`tileid`, `targetid`, `location`).
  - Primary key: `id`, a unique, arbitrary integer composed from (`tileid`, `targetid`, `location`).
  - [Column descriptions](https://data.desi.lbl.gov/doc/access/database/#fiberassign).
* `potential`. This contains a list of `targetid`s that *could* have been targeted on a given tile.
  - Loaded from the `POTENTIAL_ASSIGNMENTS` HDU in the same fiberassign files mentioned above.
  - SQLAlchemy object: `db.Potential`.
  - Unique identifier: (`tileid`, `targetid`, `location`).
  - Primary key: `id`, a unique, arbitrary integer composed from (`tileid`, `targetid`, `location`).
  - [Column descriptions](https://data.desi.lbl.gov/doc/access/database/#potential).
* `zpix`. This contains the pipeline redshifts grouped by HEALPixel.
  - Loaded from the [zcat VAC](https://data.desi.lbl.gov/doc/releases/edr/vac/zcat/): `/global/cfs/cdirs/desi/public/edr/vac/edr/zcat/fuji/v1.0/zall-pix-edr-vac.fits`.
  - Based on the `zall-pix-fuji.fits` file in the `zcatalog/` directory.
  - SQLAlchemy object: `db.Zpix`.
  - Unique identifier: (`targetid`, `survey`, `program`).
  - Primary key: `id`, a unique, arbitrary integer composed from (`targetid`, `survey`, `program`).
  - [Column descriptions](https://data.desi.lbl.gov/doc/access/database/#zpix).
* `ztile`. This contains the pipeline redshifts grouped by tile in a variety of ways.
  - Loaded from the [zcat VAC](https://data.desi.lbl.gov/doc/releases/edr/vac/zcat/): `/global/cfs/cdirs/desi/public/edr/vac/edr/zcat/fuji/v1.0/zall-tilecumulative-edr-vac.fits`.
  - Based on the `zall-tilecumulative-fuji.fits` file in the `zcatalog/` directory.
  - SQLAlchemy object: `db.Ztile`.
  - Unique identifier: (`targetid`, `spgrp`, `spgrpval`, `tileid`).
  - Primary key: `id`, a unique, arbitrary integer composed from (`targetid`, `spgrp`, `spgrpval`, `tileid`).
  - [Column descriptions](https://data.desi.lbl.gov/doc/access/database/#ztile).

### Foreign key relationships

* `fuji.target.targetid` -> `fuji.photometry.targetid`.
* `fuji.target.tileid` -> `fuji.tile.tileid`.
* `fuji.exposure.tileid` -> `fuji.tile.tileid`.
* `fuji.frame.expid` -> `fuji.exposure.expid`.
* `fuji.fiberassign.targetid` -> `fuji.photometry.targetid`.
* `fuji.fiberassign.tileid` -> `fuji.tile.tileid`.
* `fuji.potential.targetid` -> `fuji.photometry.targetid`.
* `fuji.potential.tileid` -> `fuji.tile.tileid`.
* `fuji.zpix.targetid` -> `fuji.photometry.targetid`.
* `fuji.ztile.targetphotid` -> `fuji.target.id`.
* `fuji.ztile.targetid` -> `fuji.photometry.targetid`.
* `fuji.ztile.tileid` -> `fuji.tile.tileid`.

## Authentication

Much of the functionality of Data Lab can be accessed without explicitly logging in (the service then uses an anonymous login). But some capacities, for instance saving the results of your queries to your virtual storage space, require a login (*i.e.* you will need a registered user account).

If you need to log in to Data Lab, issue this command, and respond according to the instructions:

In [ ]:
# ac.login(input("Enter user name: (+ENTER) "), getpass("Enter password: (+ENTER) "))
ac.whoAmI()

## Simple Queries

In most of the examples below, we include the equivalent raw SQL command that corresponds to the query.

### Exposures, Nights, Tiles

Here are some simple queries that demonstrate simple connections between nights, exposures and tiles.

#### How many tiles are there?

```SQL
SELECT COUNT(tileid) FROM desi_edr.tile;
```

In [ ]:
response = qc.query(sql='SELECT COUNT(tileid) FROM desi_edr.tile;', fmt='pandas', timeout=600)
response

#### On which nights were a particular tile observed?

```SQL
SELECT night, expid FROM desi_edr.exposure WHERE tileid = 100;
```

In [ ]:
response = qc.query(sql='SELECT night, expid FROM desi_edr.exposure WHERE tileid = 100;', fmt='pandas', timeout=600)
response

#### Which tiles were observed on a night?

```SQL
SELECT tileid, survey, program FROM desi_edr.exposure WHERE night = 20210115;
```

In [ ]:
response = qc.query(sql='SELECT tileid, survey, program FROM desi_edr.exposure WHERE night = 20210115;', fmt='pandas', timeout=600)
response

### Select ELG Targets

Note the special way we obtain the bitwise AND operator (`desi_mask.ELG == 2**1`).

```SQL
SELECT * from desi_edr.target WHERE (desi_target & 2) != 0;
```

In [ ]:
response = qc.query(sql='SELECT * FROM desi_edr.target WHERE (desi_target & 2) != 0 LIMIT 10;', fmt='pandas', timeout=600)
response

#### Exercise

* How many objects in the `zpix` table have `spectype` 'GALAXY'?

### Redshift and Classification

Simple query filtering on string values. Note that the slice notation `[:20]` is equivalent to `LIMIT 20` in SQL.

```SQL
SELECT spectype, subtype, z FROM desi_edr.zpix WHERE spectype = 'STAR' AND subtype != '' LIMIT 20;
```

In [ ]:
response = qc.query(sql="SELECT spectype, subtype, z FROM desi_edr.zpix WHERE spectype = 'STAR' AND subtype != '' LIMIT 20;", fmt='pandas', timeout=600)
response

## Joining Tables

### A Simple Join

Let's look at the nights and exposures on which a particular `TARGETID` was observed.

```SQL
SELECT f.tileid, e.expid, e.night FROM desi_edr.fiberassign AS f JOIN desi_edr.exposure AS e ON f.tileid = e.tileid WHERE f.targetid = 933811403620352;
```

In [ ]:
q = """SELECT f.tileid, e.expid, e.night
FROM desi_edr.fiberassign AS f
JOIN desi_edr.exposure AS e ON f.tileid = e.tileid
WHERE f.targetid = 933811403620352;"""
response = qc.query(sql=q, fmt='pandas', timeout=600)
response

### Another Simple Join

In this case, we'll look at photometric flux and measured redshift. We'll `ORDER`(`BY`) the results and `LIMIT` the query with slice notation.

```SQL
SELECT p.*, z.* FROM desi_edr.photometry AS p JOIN desi_edr.zpix AS z ON p.targetid = z.targetid ORDER BY z.z, p.flux_g LIMIT 50;
```

In [ ]:
q = """SELECT p.*, z.*
FROM desi_edr.photometry AS p
JOIN desi_edr.zpix AS z ON p.targetid = z.targetid
ORDER BY z.z, p.flux_g LIMIT 50;"""
response = qc.query(sql=q, fmt='pandas', timeout=600)
response

In [ ]:
# Avoid warnings about invalid values in np.log10().
w = (response.flux_g.values > 0) & (response.flux_r.values > 0)
g_minus_r = np.log10(response.flux_r.values[w]/response.flux_g.values[w])
r_minus_z = np.log10(response.flux_z.values[w]/response.flux_r.values[w])
redshift = np.array(response.z.values[w])
fig, axes = plt.subplots(1, 1, figsize=(5, 5), dpi=100)
p = axes.plot(g_minus_r, r_minus_z, 'k.')
foo = axes.set_xlim([-0.2, 1.0])
foo = axes.set_ylim([-0.2, 1.0])
foo = axes.set_aspect('equal')
foo = axes.set_xlabel('$g - r$')
foo = axes.set_ylabel('$r - z$')

#### Exercise

* Create a color-color plot for objects targeted as QSOs, and spectroscopically confirmed as such.

### A More Complicated Join

Let's look at objects that appear on more than one tile. For each of those tiles, how many exposures where there?

In this example, we're using `sqlalchemy.sql.func` to get the equivalent of `COUNT(*)` and a subquery that itself is a multi-table join.

```SQL
SELECT t.nexp, f.tileid, q1.targetid, q1.n_assign FROM desi_edr.fiberassign AS f
    JOIN (SELECT ff.targetid, COUNT(*) AS n_assign FROM desi_edr.fiberassign AS ff GROUP BY ff.targetid) AS q1 ON f.targetid = q1.targetid
    JOIN desi_edr.tile AS t ON f.tileid = t.tileid LIMIT 100;
```

In [ ]:
q = """SELECT t.nexp, f.tileid, q1.targetid, q1.n_assign
FROM desi_edr.fiberassign AS f
JOIN (SELECT ff.targetid, COUNT(*) AS n_assign FROM desi_edr.fiberassign AS ff GROUP BY ff.targetid) AS q1 ON f.targetid = q1.targetid
JOIN desi_edr.tile AS t ON f.tileid = t.tileid LIMIT 100;
"""
response = qc.query(sql=q, fmt='pandas', timeout=600)
response

Now let's see what the redshift table thinks are the number of exposures for these objects.

```SQL
SELECT z.* FROM desi_edr.zpix AS z WHERE z.targetid IN (6432023904256, 6448025174016[, ...]);
```

In [ ]:
q3 = """SELECT z.* FROM desi_edr.zpix AS z
WHERE z.targetid IN ({0});
""".format(', '.join(map(str, response.targetid.values.tolist())))
response3 = qc.query(sql=q3, fmt='pandas', timeout=600)
response3

#### Exercise

* What is the distribution of number of exposures?

## Matching Redshift Tables to Targeting Data

Matching redshift results back to targeting is an important task for many DESI science projects. Below we briefly describe the fundamentals of this matching using the database.

### Tile-based

The `ztile` table contains a foreign key pointing to the primary key of the `target` table. This means that for any row of `ztile`, there is a pre-matched entry in the `target` table, and it can be accessed with a simple join, such as:
```SQL
SELECT z.targetid, z.spgrp, z.spgrpval, z.tileid, z.z, z.zwarn, z.spectype,
    t.sv1_desi_target, t.sv1_bgs_target, t.sv1_mws_target, t.sv1_scnd_target,
    p.ra, p.dec
FROM desi_edr.ztile AS z
JOIN desi_edr.target AS t ON z.targetphotid = t.id
JOIN desi_edr.photometry AS p ON z.targetid = p.targetid
LIMIT 50;
```
Note the `z.targetphotid = t.id`. This query also demonstrates a JOIN with the `photometry` table, where `targetid` is unique.

In [ ]:
q = """SELECT z.targetid, z.spgrp, z.spgrpval, z.tileid, z.z, z.zwarn, z.spectype,
    t.sv1_desi_target, t.sv1_bgs_target, t.sv1_mws_target, t.sv1_scnd_target,
    p.ra, p.dec
FROM desi_edr.ztile AS z
JOIN desi_edr.target AS t ON z.targetphotid = t.id
JOIN desi_edr.photometry AS p ON z.targetid = p.targetid
LIMIT 50;"""
response = qc.query(sql=q, fmt='pandas', timeout=600)
response

### HEALPixel-based

The redshift results from HEALPixel-based coadds in the `zpix` table may be made up of several tiles, so it is not as simple to match these with targeting data. In the majority of cases, the targeting information is the same for a `targetid` across multiple tiles, as long as the tiles are part of the same survey (*e.g.* `sv1`). However it is not *always* the case, so we have instead collected the best values of the targeting information and placed those values directly in the `zpix` table. Here is the analogous query to the query above:
```SQL
SELECT z.targetid, z.survey, z.program, z.healpix, z.z, z.zwarn, z.spectype,
    z.sv1_desi_target, z.sv1_bgs_target, z.sv1_mws_target, z.sv1_scnd_target,
    p.ra, p.dec
FROM desi_edr.zpix AS z
JOIN desi_edr.photometry AS p ON z.targetid = p.targetid
LIMIT 50;
```
If you wish to explore this issue in more detail, there is an [example notebook](https://github.com/desihub/specprod-db/blob/main/doc/nb/finding-edge-cases.ipynb) (*note*: this example notebook is intended for use at [NERSC](https://www.nersc.gov)).

In [ ]:
q = """SELECT z.targetid, z.survey, z.program, z.healpix, z.z, z.zwarn, z.spectype,
    z.sv1_desi_target, z.sv1_bgs_target, z.sv1_mws_target, z.sv1_scnd_target,
    p.ra, p.dec
FROM desi_edr.zpix AS z
JOIN desi_edr.photometry AS p ON z.targetid = p.targetid
LIMIT 50;"""
response = qc.query(sql=q, fmt='pandas', timeout=600)
response

## Survey Progress

Let's see which nights have data, and count the number of exposures per night.

```SQL
SELECT e.night, COUNT(e.expid) AS n_exp FROM desi_edr.exposure AS e GROUP BY e.night ORDER BY e.night;
```

In [ ]:
q = "SELECT e.night, COUNT(e.expid) AS n_exp FROM desi_edr.exposure AS e GROUP BY e.night ORDER BY e.night;"
response = qc.query(sql=q, fmt='pandas', timeout=600)
response

Observation timestamp for a given night.  Note how we have both MJD and a corresponding `datetime.datetime` object in the database.

```SQL
SELECT e.expid, e.mjd, e.date_obs FROM desi_edr.exposure AS e WHERE e.night = 20210428 ORDER BY e.expid;
```

In [ ]:
q = "SELECT e.expid, e.mjd, e.date_obs FROM desi_edr.exposure AS e WHERE e.night = 20210428 ORDER BY e.expid;"
response = qc.query(sql=q, fmt='pandas', timeout=600)
response

So, for a given target in the `target` table, when was the observation completed?  In other words, if a target has multiple observations, we want the date of the *last* observation.  First, how many targets are there?

```SQL
SELECT COUNT(*) AS n_targets FROM desi_edr.target;
```

In [ ]:
q = "SELECT COUNT(*) AS n_targets FROM desi_edr.target;"
N_targets = qc.query(sql=q, fmt='pandas', timeout=600)
N_targets

Now we look for targets that have observations and find the MJD of the observation.

```SQL
SELECT f.targetid, e.expid, e.mjd FROM desi_edr.fiberassign AS f
    JOIN (SELECT tt.targetid FROM desi_edr.target AS tt JOIN desi_edr.fiberassign AS ff ON tt.targetid = ff.targetid
              JOIN desi_edr.exposure AS ee ON ff.tileid = ee.tileid GROUP BY tt.targetid) AS q1 ON f.targetid = q1.targetid
    JOIN desi_edr.exposure AS e ON f.tileid = e.tileid ORDER BY q1.targetid, e.expid;
```

In [ ]:
#
# Find all targetids that have observations.
#
q1 = """SELECT tt.targetid FROM desi_edr.target AS tt JOIN desi_edr.fiberassign AS ff ON tt.targetid = ff.targetid
    JOIN desi_edr.exposure AS ee ON ff.tileid = ee.tileid GROUP BY tt.targetid"""
#
# Find the exposure times for the targetids that have been observed
#
q2 = f"""SELECT f.targetid, e.expid, e.mjd FROM desi_edr.fiberassign AS f
    JOIN ({q1}) AS q1 ON f.targetid = q1.targetid
    JOIN desi_edr.exposure AS e ON f.tileid = e.tileid ORDER BY q1.targetid, e.expid;"""
response = qc.query(sql=q2, fmt='pandas', timeout=600)
targetid = response.targetid.values
expid = response.expid.values
mjd = response.mjd.values
#
# Use the counts to give the *last* observation.
#
unique_targetid, i, j, c = np.unique(targetid, return_index=True, return_inverse=True, return_counts=True)
unique_expid = expid[i + (c-1)]
unique_mjd = mjd[i + (c-1)]

Now we have the targets and the date of last observation.  But it's sorted by `targetid`.

In [ ]:
ii = unique_expid.argsort()
unique_targetid, i3, j3, c3 = np.unique(unique_expid[ii], return_index=True, return_inverse=True, return_counts=True)
N_completed = np.cumsum(c3)

In [ ]:
min_mjd = 10*(int(mjd.min())//10)
fig, axes = plt.subplots(1, 1, figsize=(8, 8), dpi=100)
p1 = axes.plot(unique_mjd[ii][i3] - min_mjd, N_completed/N_targets.n_targets.values[0], 'k-')
foo = axes.set_xlabel(f'MJD - {min_mjd:d}')
foo = axes.set_ylabel('Fraction completed')
foo = axes.grid(True)
# foo = axes.legend(loc=1)

### Exercise

* Break down the progress by target class, target bit, etc.

## Using q3c

[q3c](https://github.com/segasai/q3c) ([Koposov & Bartunov 2006](https://ui.adsabs.harvard.edu/abs/2006ASPC..351..735K/abstract)) is a popular library that provides spatial indexing and searching in astronomical databases. Here we'll demonstrate how to access this functionality. This is a radial ("cone") search on an arbitrary point in the DESI footprint:

```SQL
SELECT p.*, z.*, q3c_dist(p.ra, p.dec, 180.0, 0.0) AS radial_distance
    FROM desi_edr.photometry AS p JOIN desi_edr.zpix AS z ON p.targetid = z.targetid
    WHERE q3c_radial_query(p.ra, p.dec, 180.0, 0.0, 1.0/60.0); -- 1 arcmin
```

In [ ]:
q = """SELECT p.*, z.*, q3c_dist(p.ra, p.dec, 180.0, 0.0) AS radial_distance
    FROM desi_edr.photometry AS p JOIN desi_edr.zpix AS z ON p.targetid = z.targetid
    WHERE q3c_radial_query(p.ra, p.dec, 180.0, 0.0, 1.0/60.0);"""
response = qc.query(sql=q, fmt='pandas', timeout=600)
response

### Exercise

* What spectra are near your favourite object?

## Resources and References

* [DESI Database Notebook](https://github.com/desihub/tutorials/blob/main/database/spectroscopic-production-database.ipynb) (*note*: requires access to [NERSC](https://www.nersc.gov)).
* [DESI Database Documentation](https://data.desi.lbl.gov/doc/access/database/).
* [DESI Early Data Release Paper](https://arxiv.org/abs/2306.06308).